<a href="https://colab.research.google.com/github/nikunjchapadia/fastbook/blob/master/nlp_fscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 4.4 MB/s 
     |████████████████████████████████| 189 kB 42.1 MB/s 
     |████████████████████████████████| 48 kB 4.4 MB/s 
     |████████████████████████████████| 1.2 MB 33.6 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 558 kB 36.8 MB/s 
     |████████████████████████████████| 51 kB 241 kB/s 
     |████████████████████████████████| 130 kB 44.2 MB/s 
Mounted at /content/gdrive


In [ ]:
from fastbook import *

In [ ]:
from fastai.text.all import *
# first 10,000 numbers reterun in english 
path = untar_data(URLs.HUMAN_NUMBERS)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

(#2) [Path('valid.txt'),Path('train.txt')]

In [ ]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

In [ ]:
text = ' . '.join([l.strip() for l in lines])
text[:100]

'one . two . three . four . five . six . seven . eight . nine . ten . eleven . twelve . thirteen . fo'

In [ ]:
tokens = text.split(' ')
tokens[:10]

['one', '.', 'two', '.', 'three', '.', 'four', '.', 'five', '.']

In [ ]:
vocab = L(*tokens).unique()
vocab

(#30) ['one','.','two','three','four','five','six','seven','eight','nine'...]

In [ ]:
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
tokens, nums

(['one',
  '.',
  'two',
  '.',
  'three',
  '.',
  'four',
  '.',
  'five',
  '.',
  'six',
  '.',
  'seven',
  '.',
  'eight',
  '.',
  'nine',
  '.',
  'ten',
  '.',
  'eleven',
  '.',
  'twelve',
  '.',
  'thirteen',
  '.',
  'fourteen',
  '.',
  'fifteen',
  '.',
  'sixteen',
  '.',
  'seventeen',
  '.',
  'eighteen',
  '.',
  'nineteen',
  '.',
  'twenty',
  '.',
  'twenty',
  'one',
  '.',
  'twenty',
  'two',
  '.',
  'twenty',
  'three',
  '.',
  'twenty',
  'four',
  '.',
  'twenty',
  'five',
  '.',
  'twenty',
  'six',
  '.',
  'twenty',
  'seven',
  '.',
  'twenty',
  'eight',
  '.',
  'twenty',
  'nine',
  '.',
  'thirty',
  '.',
  'thirty',
  'one',
  '.',
  'thirty',
  'two',
  '.',
  'thirty',
  'three',
  '.',
  'thirty',
  'four',
  '.',
  'thirty',
  'five',
  '.',
  'thirty',
  'six',
  '.',
  'thirty',
  'seven',
  '.',
  'thirty',
  'eight',
  '.',
  'thirty',
  'nine',
  '.',
  'forty',
  '.',
  'forty',
  'one',
  '.',
  'forty',
  'two',
  '.',
  'forty',
  't

In [ ]:
L((tokens[i:i+3], tokens[i+3]) for i in range(0,len(tokens)-4,3))
# independent variable - dependent variables
# ['one', '.', 'two'], '.'

(#21031) [(['one', '.', 'two'], '.'),(['.', 'three', '.'], 'four'),(['four', '.', 'five'], '.'),(['.', 'six', '.'], 'seven'),(['seven', '.', 'eight'], '.'),(['.', 'nine', '.'], 'ten'),(['ten', '.', 'eleven'], '.'),(['.', 'twelve', '.'], 'thirteen'),(['thirteen', '.', 'fourteen'], '.'),(['.', 'fifteen', '.'], 'sixteen')...]

In [ ]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

(#21031) [(tensor([0, 1, 2]), 1),(tensor([1, 3, 1]), 4),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 7),(tensor([7, 1, 8]), 1),(tensor([1, 9, 1]), 10),(tensor([10,  1, 11]), 1),(tensor([ 1, 12,  1]), 13),(tensor([13,  1, 14]), 1),(tensor([ 1, 15,  1]), 16)...]

In [ ]:
bs = 64
cut = int(len(seqs) * 0.8) # first 80% training set rest 20% for validation set
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

In [ ]:
# create neural netwrok architecture 
# 3 layers 
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

In [ ]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.824297,1.970941,0.467554,00:06
1,1.386973,1.823242,0.467554,00:04
2,1.417556,1.654498,0.494414,00:03
3,1.376440,1.650849,0.494414,00:03


In [ ]:
n,counts = 0,torch.zeros(len(vocab))
for x,y in dls.valid:
    n += y.shape[0]
    for i in range_of(vocab): counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)
idx, vocab[idx.item()], counts[idx].item()/n

(tensor(29), 'thousand', 0.15165200855716662)

In [ ]:
c = Counter(tokens[cut:])
mc = c.most_common(10)
mc

[('thousand', 7104),
 ('.', 7103),
 ('hundred', 6405),
 ('nine', 2440),
 ('eight', 2344),
 ('five', 2340),
 ('six', 2340),
 ('seven', 2340),
 ('three', 2339),
 ('four', 2339)]

In [ ]:
mc[0][1], len(tokens[cut:])
mc[0][1]/len(tokens[cut:])
# if we have predicted every number is thousand as it appers more tha any other we will be 15% accurate 
# compare to our model which is close to 50% 

0.15353028894988222

In [ ]:
# ('.', 7103)
mc[1][1], len(tokens[cut:])
mc[1][1]/len(tokens[cut:])

0.15350867714118996

In [ ]:
# ('hundred', 6405)
mc[2][1], len(tokens[cut:])
mc[2][1]/len(tokens[cut:])

0.13842363467398586

In [ ]:
# ('seven', 2340)
mc[7][1], len(tokens[cut:])
mc[7][1]/len(tokens[cut:])

0.05057163233991053

In [ ]:
# Recurrent Neural Network

In [ ]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            # print("-------------------", i, x[:i])
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [ ]:
learn = Learner(dls, LMModel2(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.823478,1.926430,0.467792,00:02
1,1.402421,1.814042,0.468980,00:02
2,1.418952,1.683436,0.494652,00:02
3,1.390207,1.662460,0.460661,00:02


In [ ]:
# improving rnn

class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        # h persists from batch to batch we need to reset it for each batch
        # gradient history will be reset - back propogation through time BPTT 
        self.h = self.h.detach() 
        return out
    # at each epoch we are going back to our natural numbers so set to 0
    def reset(self): self.h = 0

In [ ]:
m = len(seqs)//bs
m,bs,len(seqs)

(328, 64, 21031)

In [ ]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [ ]:
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

In [ ]:
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.722347,1.807805,0.483654,00:02
1,1.270391,1.752513,0.455048,00:02
2,1.091535,1.593707,0.516106,00:02
3,1.017162,1.575380,0.499279,00:02
4,0.989237,1.612636,0.521154,00:02
5,0.929354,1.645421,0.552885,00:03
6,0.866206,1.588576,0.581731,00:02
7,0.834293,1.633378,0.585337,00:02
8,0.793425,1.750832,0.574038,00:02
9,0.780956,1.721157,0.582452,00:02


In [ ]:
# instead of just predicting 4 word based on first 3 we will try to predict any word
# we will offset sequence by 1 for dependent variable and indenpendent variable
# 1 2 3 4 5 
# 2 3 4 5 6 
sl = 16
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [ ]:
[L(vocab[o] for o in s) for s in seqs[0]]

[(#16) ['one','.','two','.','three','.','four','.','five','.'...],
 (#16) ['.','two','.','three','.','four','.','five','.','six'...]]

In [ ]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

In [ ]:
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [ ]:
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.226497,3.176609,0.157471,00:01
1,2.504166,2.080834,0.468750,00:01
2,1.842446,1.935876,0.467041,00:01
3,1.512856,1.881448,0.499512,00:01
4,1.333710,1.794642,0.549316,00:01
5,1.184813,1.909202,0.558594,00:01
6,1.073470,1.926587,0.571777,00:01
7,0.954470,1.941545,0.611247,00:01
8,0.855734,1.933198,0.636068,00:01
9,0.780780,1.906064,0.635742,00:01


In [ ]:
# Multilayers or stacked RNN
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [ ]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.989784,2.513293,0.437093,00:01
1,2.123006,1.772213,0.470947,00:01
2,1.692048,1.909070,0.319336,00:01
3,1.464266,1.724511,0.486816,00:01
4,1.309704,1.908625,0.488363,00:01
5,1.165450,2.173341,0.489827,00:01
6,1.039468,2.296848,0.494141,00:01
7,0.941451,2.373549,0.493652,00:01
8,0.845794,2.388737,0.501709,00:01
9,0.763576,2.339804,0.512533,00:01


In [ ]:
# results are not good as we got before 
# we have very deep model - which is hard to train 
# we get exploding or disappearing activations  
# explosion is when number becomes too big 
# disappearing is when number becomes too small 

In [ ]:
# solution use architecture LSTM

class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.forget_gate = nn.Linear(ni + nh, nh)
        self.input_gate  = nn.Linear(ni + nh, nh)
        self.cell_gate   = nn.Linear(ni + nh, nh)
        self.output_gate = nn.Linear(ni + nh, nh)

    def forward(self, input, state):
        h,c = state
        h = torch.cat([h, input], dim=1)
        forget = torch.sigmoid(self.forget_gate(h))
        c = c * forget
        inp = torch.sigmoid(self.input_gate(h))
        cell = torch.tanh(self.cell_gate(h))
        c = c + inp * cell
        out = torch.sigmoid(self.output_gate(h))
        h = out * torch.tanh(c)
        return h, (h,c)

In [ ]:
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.ih = nn.Linear(ni,4*nh)
        self.hh = nn.Linear(nh,4*nh)

    def forward(self, input, state):
        h,c = state
        # One big multiplication for all the gates is better than 4 smaller ones
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)
        ingate,forgetgate,outgate = map(torch.sigmoid, gates[:3])
        cellgate = gates[3].tanh()

        c = (forgetgate*c) + (ingate*cellgate)
        h = outgate * c.tanh()
        return h, (h,c)

In [ ]:
t = torch.arange(0,10); t

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
t.chunk(2)

(tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]))

In [ ]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.997656,2.740257,0.427979,00:02
1,2.190678,1.936098,0.321370,00:02
2,1.618659,1.815960,0.480062,00:02
3,1.313747,2.088989,0.529216,00:02
4,1.050850,1.999131,0.617676,00:02
5,0.718113,1.642511,0.737142,00:02
6,0.407751,1.448469,0.753662,00:02
7,0.210455,1.317910,0.759115,00:02
8,0.111644,1.164466,0.790039,00:02
9,0.065864,1.110025,0.807943,00:02


In [ ]:
# Regularizing an LSTM
# to ensuer we dont overfit 
# Dropout - regularization technique
# randomly delete activation on each batch 

class Dropout(Module):
    def __init__(self, p): self.p = p
    def forward(self, x):
        if not self.training: return x
        mask = x.new(*x.shape).bernoulli_(1-p)
        return x * mask.div_(1-p)

In [ ]:
# Activation Regularization and Temporal Activation Regularization

In [ ]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
learn = Learner(dls, LMModel7(len(vocab), 64, 2, 0.5),
                loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                cbs=[ModelResetter, RNNRegularizer(alpha=2, beta=1)])

In [ ]:
learn = TextLearner(dls, LMModel7(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.012636,0.666969,0.842285,00:02
1,0.027867,0.663418,0.819417,00:02
2,0.035776,1.032247,0.806152,00:02
3,0.040214,0.813783,0.850342,00:02
4,0.028341,0.650990,0.852295,00:02
5,0.022422,0.739949,0.833496,00:02
6,0.022070,0.780746,0.844157,00:02
7,0.017627,0.759494,0.842285,00:02
8,0.015892,0.656404,0.849935,00:02
9,0.013619,0.746668,0.848470,00:02
